In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


File used for preprocessing of mESC_Ag.

In [2]:
import pandas as pd
import numpy as np
import re
from matplotlib import pyplot as plt
from matplotlib import colors
from matplotlib.ticker import PercentFormatter
from sklearn.model_selection import train_test_split
import os

In [3]:
Root_Folder = "/content/drive/MyDrive"
data_parent_folder = "/Data"

In [4]:
gastr_ind_drp = [0,1,2,3,4,8,9,10]
gastr_QC = pd.read_csv(Root_Folder + data_parent_folder + "/Data_Gastr/sample_metadata_gastr.csv",'\t',index_col=[0])

gastr_QC = gastr_QC.drop(columns=gastr_QC.columns[gastr_ind_drp],axis=1)
gastr_QC = gastr_QC.loc[gastr_QC['pass_rnaQC'] & gastr_QC['pass_metQC'] & gastr_QC['pass_accQC'] ,: ]
gastr_QC = gastr_QC.index.str.replace('-','.')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only
  exec(code_obj, self.user_global_ns, self.user_ns)


In [5]:
print(gastr_QC)

Index(['E4.5.5.5_new_Plate3_E09', 'E4.5.5.5_new_Plate3_H09',
       'E4.5.5.5_new_Plate4_F01', 'E4.5.5.5_new_Plate4_C01',
       'E4.5.5.5_new_Plate4_E01', 'E4.5.5.5_new_Plate4_G01',
       'E4.5.5.5_new_Plate4_D01', 'E4.5.5.5_new_Plate4_A01',
       'E4.5.5.5_new_Plate4_H01', 'E4.5.5.5_new_Plate4_B01',
       ...
       'PS_VE_Plate1_F06', 'PS_VE_Plate1_A06', 'PS_VE_Plate1_C06',
       'PS_VE_Plate1_B06', 'PS_VE_Plate1_H06', 'PS_VE_Plate1_D04',
       'PS_VE_Plate1_A04', 'PS_VE_Plate1_C04', 'PS_VE_Plate1_B04',
       'PS_VE_Plate1_H04'],
      dtype='object', name='sample', length=826)


In [6]:
def stand_sort_dat(Met,CpG,Acc,GpC,Rna,Rna_nrm):

        gene_com_ind = Rna.index.intersection(CpG.index)
        gene_com_ind = GpC.index.intersection(gene_com_ind)

        gene_met_ind = CpG.index.symmetric_difference(gene_com_ind)
        gene_acc_ind = GpC.index.symmetric_difference(gene_com_ind)
        gene_rna_ind = Rna.index.symmetric_difference(gene_com_ind)
        
        gene_all = np.union1d(gene_com_ind,gene_met_ind)
        gene_all = np.union1d(gene_all,gene_acc_ind)
        gene_all = np.union1d(gene_all,gene_rna_ind)

        cell_com_ind = Rna.columns.intersection(CpG.columns)
        cell_com_ind = GpC.columns.intersection(cell_com_ind)

        cell_met_ind=CpG.columns.symmetric_difference(cell_com_ind)
        cell_acc_ind=GpC.columns.symmetric_difference(cell_com_ind)
        cell_rna_ind=Rna.columns.symmetric_difference(cell_com_ind)

        cell_all = np.union1d(cell_com_ind,cell_met_ind)
        cell_all = np.union1d(cell_all,cell_acc_ind)
        cell_all = np.union1d(cell_all,cell_rna_ind)

        Met_fin = pd.DataFrame(-1, index=gene_all, columns=cell_all)
        CpG_fin = pd.DataFrame(-1, index=gene_all, columns=cell_all)
        Acc_fin = pd.DataFrame(-1, index=gene_all, columns=cell_all)
        GpC_fin = pd.DataFrame(-1, index=gene_all, columns=cell_all)  
        Rna_fin = pd.DataFrame(0, index=gene_all, columns=cell_all)
        Rna_nrm_fin = pd.DataFrame(0.00001, index=["norm_fact"], columns=cell_all)

        Met_fin.loc[Met.index,Met.columns]=Met
        CpG_fin.loc[CpG.index,CpG.columns]=CpG
        
        Acc_fin.loc[Acc.index,Acc.columns]= Acc
        GpC_fin.loc[GpC.index,GpC.columns] = GpC

        Rna_fin.loc[Rna.index,Rna.columns]=Rna
        Rna_nrm_fin[Rna_nrm.index] = np.asarray(Rna_nrm.iloc[:,0])

        Met_fin=Met_fin.replace(-1,0)
        CpG_fin=CpG_fin.replace(-1,0)
        Acc_fin=Acc_fin.replace(-1,0)
        GpC_fin=GpC_fin.replace(-1,0)
        Rna_fin=Rna_fin.replace(-1,0)
        
        return Met_fin,CpG_fin,Acc_fin,GpC_fin,Rna_fin,Rna_nrm_fin

In [7]:
Met_gastr=pd.read_csv(Root_Folder + data_parent_folder + "/Raw_Data_Gastr/MT_gastrulation_prom_met.csv","\t",index_col=[0])
CpG_gastr=pd.read_csv(Root_Folder + data_parent_folder + "/Raw_Data_Gastr/MT_gastrulation_prom_cpg.csv","\t",index_col=[0])

Acc_gastr=pd.read_csv(Root_Folder + data_parent_folder + "/Raw_Data_Gastr/NT_gastrulation_prom_met.csv","\t",index_col=[0])
GpC_gastr=pd.read_csv(Root_Folder + data_parent_folder + "/Raw_Data_Gastr/NT_gastrulation_prom_gpc.csv","\t",index_col=[0])

Rna_gastr=pd.read_csv(Root_Folder + data_parent_folder + "/Raw_Data_Gastr/MT_gastrulation_prom_rna.csv",'\t',index_col=[0])
Rna_nrm_gastr = pd.read_csv(Root_Folder + data_parent_folder + "/Raw_Data_Gastr/MT_gastrulation_prom_nrm.csv",'\t',index_col=[1])

Met_gastr.columns = Met_gastr.columns.str.replace('-','.')
CpG_gastr.columns = CpG_gastr.columns.str.replace('-','.')

Acc_gastr.columns = Acc_gastr.columns.str.replace('-','.')
GpC_gastr.columns = GpC_gastr.columns.str.replace('-','.')
Rna_nrm_gastr.index = Rna_nrm_gastr.index.str.replace('-','.')

Met_gastr.columns = Met_gastr.columns.str.replace('.tsv.gz','')
CpG_gastr.columns = CpG_gastr.columns.str.replace('.tsv.gz','')

Acc_gastr.columns = Acc_gastr.columns.str.replace('.tsv.gz','')
GpC_gastr.columns = GpC_gastr.columns.str.replace('.tsv.gz','')

Met_gastr,CpG_gastr,Acc_gastr,GpC_gastr,Rna_gastr,Rna_nrm_gastr = stand_sort_dat(Met_gastr,CpG_gastr,Acc_gastr,GpC_gastr,Rna_gastr,Rna_nrm_gastr)

Met_gastr_pq = Met_gastr
CpG_gastr_pq = CpG_gastr
Acc_gastr_pq = Acc_gastr
GpC_gastr_pq = GpC_gastr
Rna_gastr_pq = Rna_gastr
Rna_nrm_gastr_pq = Rna_nrm_gastr

gastr_QC = np.intersect1d(gastr_QC,Met_gastr.columns)
gastr_QC = np.intersect1d(gastr_QC,Acc_gastr.columns)
gastr_QC = np.intersect1d(gastr_QC,Rna_gastr.columns)

Met_gastr = Met_gastr[gastr_QC]
CpG_gastr = CpG_gastr[gastr_QC]
Acc_gastr = Acc_gastr[gastr_QC]
GpC_gastr = GpC_gastr[gastr_QC]
Rna_gastr = Rna_gastr[gastr_QC]
Rna_nrm_gastr = Rna_nrm_gastr[gastr_QC]

Met_gastr_pq.drop(gastr_QC,axis=1,inplace=True)
CpG_gastr_pq.drop(gastr_QC,axis=1,inplace=True)
Acc_gastr_pq.drop(gastr_QC,axis=1,inplace=True)
GpC_gastr_pq.drop(gastr_QC,axis=1,inplace=True)
Rna_gastr_pq.drop(gastr_QC,axis=1,inplace=True)
Rna_nrm_gastr_pq.drop(gastr_QC,axis=1,inplace=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: FutureWarning: The default value of regex will change from True to False in a future version.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: FutureWarning: The default value of regex will change from True to False in a future version.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: FutureWarning: The default value of regex will change from True to False in a future version.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: FutureWarning: The default value of regex will change from True to False in a future version.


In [8]:
Met = Met_gastr
CpG = CpG_gastr
Acc = Acc_gastr
GpC = GpC_gastr
Rna = Rna_gastr
Rna_nrm = Rna_nrm_gastr

Met=Met.T
CpG=CpG.T
Acc=Acc.T
GpC=GpC.T
Rna=Rna.T
Rna_nrm = Rna_nrm.T

In [9]:
Met_pq = Met_gastr_pq
CpG_pq = CpG_gastr_pq
Acc_pq = Acc_gastr_pq
GpC_pq = GpC_gastr_pq
Rna_pq = Rna_gastr_pq
Rna_nrm_pq = Rna_nrm_gastr_pq


Met_pq=Met_pq.T
CpG_pq=CpG_pq.T
Acc_pq=Acc_pq.T
GpC_pq=GpC_pq.T
Rna_pq=Rna_pq.T
Rna_nrm_pq=Rna_nrm_pq.T

ind_pq = np.intersect1d(Met.columns,Met_pq.columns)
Met_pq = Met_pq.loc[:,ind_pq]
CpG_pq = CpG_pq.loc[:,ind_pq]
Acc_pq = Acc_pq.loc[:,ind_pq]
GpC_pq = GpC_pq.loc[:,ind_pq]
Rna_pq = Rna_pq.loc[:,ind_pq]

In [10]:
#QC
CpG_ind_gene = np.mean(CpG==0,axis=0)
GpC_ind_gene = np.mean(GpC==0,axis=0)
Rna_ind_gene = np.mean(Rna==0,axis=0)

ind_drp_cpg_gene = np.where(CpG_ind_gene > 0.65) #0.65
ind_drp_gpc_gene = np.where(GpC_ind_gene > 0.65) #0.65
ind_drp_rna_gene = np.where(Rna_ind_gene > 0.65) #0.65

ind_drp_gene = np.union1d(ind_drp_cpg_gene,ind_drp_rna_gene)
ind_drp_gene = np.union1d(ind_drp_gene,ind_drp_gpc_gene)

In [11]:
print(ind_drp_gene)

[    1     2     3 ... 18845 18846 18847]


In [12]:
Met.drop(Met.columns[ind_drp_gene],axis=1,inplace=True)
CpG.drop(CpG.columns[ind_drp_gene],axis=1,inplace=True)
Acc.drop(Acc.columns[ind_drp_gene],axis=1,inplace=True)
GpC.drop(GpC.columns[ind_drp_gene],axis=1,inplace=True)
Rna.drop(Rna.columns[ind_drp_gene],axis=1,inplace=True)

Met_pq.drop(Met_pq.columns[ind_drp_gene],axis=1,inplace=True)
CpG_pq.drop(CpG_pq.columns[ind_drp_gene],axis=1,inplace=True)
Acc_pq.drop(Acc_pq.columns[ind_drp_gene],axis=1,inplace=True)
GpC_pq.drop(GpC_pq.columns[ind_drp_gene],axis=1,inplace=True)
Rna_pq.drop(Rna_pq.columns[ind_drp_gene],axis=1,inplace=True)

In [14]:
cell_ind=Met.index
gn_ind = Met.columns
train_cells, test_cells=train_test_split(cell_ind, test_size=0.23,shuffle=True)

Met_train=Met.loc[train_cells,:]
Met_test=Met.loc[test_cells,:]

CpG_train=CpG.loc[train_cells,:]
CpG_test=CpG.loc[test_cells,:]

Acc_train=Acc.loc[train_cells,:]
Acc_test=Acc.loc[test_cells,:]

GpC_train=GpC.loc[train_cells,:]
GpC_test=GpC.loc[test_cells,:]

Rna_train=Rna.loc[train_cells,:]
Rna_test=Rna.loc[test_cells,:]

Rna_nrm_train=Rna_nrm.loc[train_cells,:]
Rna_nrm_test=Rna_nrm.loc[test_cells,:]

In [15]:
gn_ind = pd.DataFrame(gn_ind,columns = ['ens_id'])
print(gn_ind)

                  ens_id
0     ENSMUSG00000000001
1     ENSMUSG00000000056
2     ENSMUSG00000000078
3     ENSMUSG00000000085
4     ENSMUSG00000000088
...                  ...
6326  ENSMUSG00000106864
6327  ENSMUSG00000106918
6328  ENSMUSG00000107283
6329  ENSMUSG00000109336
6330  ENSMUSG00000109865

[6331 rows x 1 columns]


In [16]:
if os.path.isdir(Root_Folder + data_parent_folder + "/Data_Gastr") == False:
        os.mkdir(Root_Folder + data_parent_folder + "/Data_Gastr")

Met_test.to_csv(Root_Folder + data_parent_folder + "/Data_Gastr/Met_test_gastr.csv", header=True)
Met_train.to_csv(Root_Folder + data_parent_folder + "/Data_Gastr/Met_train_gastr.csv", header=True)

CpG_test.to_csv(Root_Folder + data_parent_folder + "/Data_Gastr/CpG_test_gastr.csv", header=True)
CpG_train.to_csv(Root_Folder + data_parent_folder + "/Data_Gastr/CpG_train_gastr.csv", header=True)

Acc_test.to_csv(Root_Folder + data_parent_folder + "/Data_Gastr/Acc_test_gastr.csv", header=True)
Acc_train.to_csv(Root_Folder + data_parent_folder + "/Data_Gastr/Acc_train_gastr.csv", header=True)

GpC_test.to_csv(Root_Folder + data_parent_folder + "/Data_Gastr/GpC_test_gastr.csv", header=True)
GpC_train.to_csv(Root_Folder + data_parent_folder + "/Data_Gastr/GpC_train_gastr.csv", header=True)

Rna_test.to_csv(Root_Folder + data_parent_folder + "/Data_Gastr/Rna_test_gastr.csv", header=True)
Rna_train.to_csv(Root_Folder + data_parent_folder + "/Data_Gastr/Rna_train_gastr.csv", header=True)

Rna_nrm_test.to_csv(Root_Folder + data_parent_folder + "/Data_Gastr/Rna_nrm_test_gastr.csv", header=True)
Rna_nrm_train.to_csv(Root_Folder + data_parent_folder + "/Data_Gastr/Rna_nrm_train_gastr.csv", header=True)

'\nif os.path.isdir(Root_Folder + data_parent_folder + "/Data_Gastr") == False:\n        os.mkdir(Root_Folder + data_parent_folder + "/Data_ATAC")\n\nMet_test.to_csv(Root_Folder + data_parent_folder + "/Data_Gastr/Met_test_gastr.csv", header=True)\nMet_train.to_csv(Root_Folder + data_parent_folder + "/Data_Gastr/Met_train_gastr.csv", header=True)\n\nCpG_test.to_csv(Root_Folder + data_parent_folder + "/Data_Gastr/CpG_test_gastr.csv", header=True)\nCpG_train.to_csv(Root_Folder + data_parent_folder + "/Data_Gastr/CpG_train_gastr.csv", header=True)\n\nAcc_test.to_csv(Root_Folder + data_parent_folder + "/Data_Gastr/Acc_test_gastr.csv", header=True)\nAcc_train.to_csv(Root_Folder + data_parent_folder + "/Data_Gastr/Acc_train_gastr.csv", header=True)\n\nGpC_test.to_csv(Root_Folder + data_parent_folder + "/Data_Gastr/GpC_test_gastr.csv", header=True)\nGpC_train.to_csv(Root_Folder + data_parent_folder + "/Data_Gastr/GpC_train_gastr.csv", header=True)\n\nRna_test.to_csv(Root_Folder + data_parent_